In [3]:
import io
import pandas as pd
import requests

In [5]:
df = pd.read_csv("yellow_tripdata_2024-12.csv")

/var/folders/h1/qm70dqtj4db186bg7jxn27nw0000gn/T/ipykernel_70611/2713937671.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("yellow_tripdata_2024-12.csv")


In [6]:
# 1. timestamps
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"], errors="coerce")
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], errors="coerce")
df = df.dropna(subset=["tpep_pickup_datetime", "tpep_dropoff_datetime"])

# remove nonsense trips: dropoff before pickup
df = df[df["tpep_dropoff_datetime"] >= df["tpep_pickup_datetime"]]

# 2. passenger count (keep between 1–6)
df = df[(df["passenger_count"] >= 1) & (df["passenger_count"] <= 6)]

# 3. trip distance (remove zero/negative, filter huge outliers >100 miles)
df = df[(df["trip_distance"] > 0) & (df["trip_distance"] <= 100)]

# 4. fares and amounts (remove negatives)
monetary_cols = ["fare_amount","extra","mta_tax","tip_amount",
                 "tolls_amount","improvement_surcharge",
                 "total_amount","congestion_surcharge","Airport_fee"]
for col in monetary_cols:
    df = df[df[col] >= 0]

# 5. remove duplicates
df = df.drop_duplicates()

# save cleaned CSV
df.to_csv("yellow_tripdata_2024-12_clean.csv", index=False)

In [7]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [8]:
df.tail()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
3201138,2,2024-12-31 22:56:33,2024-12-31 23:05:57,1.0,1.90,1.0,N,237,262,1,12.1,1.0,0.5,1.88,0.0,1.0,18.98,2.5,0.00,3201138
3201139,2,2024-12-31 23:18:37,2024-12-31 23:32:32,1.0,3.88,1.0,N,237,226,2,19.1,1.0,0.5,0.00,0.0,1.0,24.10,2.5,0.00,3201139
3201140,2,2024-12-31 23:08:52,2024-12-31 23:34:59,1.0,5.53,1.0,N,142,255,1,29.6,1.0,0.5,2.00,0.0,1.0,36.60,2.5,0.00,3201140
3201141,2,2024-12-31 23:15:55,2024-12-31 23:24:24,1.0,0.89,1.0,N,43,48,1,9.3,1.0,0.5,2.14,0.0,1.0,16.44,2.5,0.00,3201141
3201142,2,2024-12-31 23:13:22,2024-12-31 23:49:57,1.0,20.49,1.0,N,132,33,1,77.9,1.0,0.5,10.00,0.0,1.0,92.15,0.0,1.75,3201142


In [9]:
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday


datetime_dim['datetime_id'] = datetime_dim.index

# datetime_dim = datetime_dim.rename(columns={'tpep_pickup_datetime': 'datetime_id'}).reset_index(drop=True)
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]
#
datetime_dim.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2024-12-01 00:12:27,0,1,12,2024,6,2024-12-01 00:31:12,0,1,12,2024,6
1,1,2024-11-30 23:56:04,23,30,11,2024,5,2024-12-01 00:28:15,0,1,12,2024,6
2,2,2024-12-01 00:50:35,0,1,12,2024,6,2024-12-01 01:24:46,1,1,12,2024,6
3,3,2024-12-01 00:18:16,0,1,12,2024,6,2024-12-01 00:33:16,0,1,12,2024,6
4,4,2024-12-01 00:56:13,0,1,12,2024,6,2024-12-01 01:18:25,1,1,12,2024,6


In [10]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

In [11]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

In [12]:
rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,2.0,JFK
3,3,1.0,Standard rate
4,4,1.0,Standard rate


In [13]:
# Create pickup location dimension table
pickup_location_dim = df[['PULocationID']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id', 'PULocationID']]

# Create dropoff location dimension table
dropoff_location_dim = df[['DOLocationID']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id', 'DOLocationID']]

In [14]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [15]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [16]:
fact_table.columns

Index(['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
       'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
       'pickup_location_id', 'dropoff_location_id', 'payment_type_id',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

##### fact_table

In [17]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,0,0,38.0,6.0,0.5,4.72,0.00,1.0,51.97
1,1,2,1,1,1,1,N,1,1,1,37.3,1.0,0.5,8.46,0.00,1.0,50.76
2,2,2,2,2,2,2,N,2,2,2,70.0,0.0,0.5,0.00,6.94,1.0,82.69
3,3,2,3,3,3,3,N,3,3,3,15.6,1.0,0.5,4.12,0.00,1.0,24.72
4,4,2,4,4,4,4,N,4,4,4,26.8,1.0,0.5,5.00,0.00,1.0,36.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3201138,3201138,2,3201138,3201138,3201138,3201138,N,3201138,3201138,3201138,12.1,1.0,0.5,1.88,0.00,1.0,18.98
3201139,3201139,2,3201139,3201139,3201139,3201139,N,3201139,3201139,3201139,19.1,1.0,0.5,0.00,0.00,1.0,24.10
3201140,3201140,2,3201140,3201140,3201140,3201140,N,3201140,3201140,3201140,29.6,1.0,0.5,2.00,0.00,1.0,36.60
3201141,3201141,2,3201141,3201141,3201141,3201141,N,3201141,3201141,3201141,9.3,1.0,0.5,2.14,0.00,1.0,16.44
